In [1]:
import pyspark
import pandas as pd
import os
from google.cloud import bigquery
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as f

In [2]:
#  pip install google-cloud-bigquery

In [3]:
df_2023 = pd.read_csv('https://raw.githubusercontent.com/Williamsrandrew86/SOL-data-project/main/data/school_pass_rates_by_test_2022_2023.csv')

In [4]:
df_2023

,LEVEL,Div Num,Div Name,Sch Num,Sch Name,Sch Type,Low Grade,High Grade,Subject,Grade,Test,2020-2021 Pass Rate,2021-2022 Pass Rate,2022-2023 Pass Rate,2020-2021 Adv Pass Rate,2021-2022 Adv Pass Rate,2022-2023 Adv Pass Rate
0,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5.0,English: Reading,Gr 3,English Reading,50,67,65,3,7,5
1,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5.0,English: Reading,Gr 4,English Reading,56,67,62,6,11,9
2,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5.0,English: Reading,Gr 5,English Reading,45,68,67,7,7,3
3,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5.0,History and Social Sciences,Gr 5,History,<,NaN,NaN,<,NaN,NaN
4,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5.0,History and Social Sciences,Content Specific,VA Studies,NaN,72,65,NaN,21,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26603,SCH,98,York County,60,Yorktown Middle,Middle,6,8.0,Remote Mathematics,Gr 6,Mathematics,<,NaN,NaN,<,NaN,NaN
26604,SCH,98,York County,60,Yorktown Middle,Middle,6,8.0,Remote Mathematics,Gr 7,Mathematics,<,NaN,NaN,<,NaN,NaN
26605,SCH,98,York County,60,Yorktown Middle,Middle,6,8.0,Remote Mathematics,Gr 8,Mathematics,<,NaN,NaN,<,NaN,NaN
26606,SCH,98,York County,60,Yorktown Middle,Middle,6,8.0,Remote Science,Gr 8,Science,<,NaN,NaN,<,NaN,NaN


In [5]:
mod_df_2023 = df_2023.drop(['2020-2021 Pass Rate','2021-2022 Pass Rate','2020-2021 Adv Pass Rate','2021-2022 Adv Pass Rate'], axis=1)

In [6]:
mod_df_2023.columns = mod_df_2023.columns.str.replace(' ', '_') #change the spaces with underscores
mod_df_2023.columns = mod_df_2023.columns.str.replace('-', '_') #change the hypens with underscores

In [7]:
# Specify columns to replace NaN with -9999
columns_to_replace = ['High_Grade']

# Replace NaN values with -9999 for specific columnsd
mod_df_2023[columns_to_replace] = mod_df_2023[columns_to_replace].fillna(value=-9999)

In [8]:
# Specify columns to replace NaN with -9999
columns_to_replace = ['Low_Grade']

# Replace NaN values with None for specific columns
mod_df_2023[columns_to_replace] = mod_df_2023[columns_to_replace].fillna(value='None')

In [9]:
#['2022-2023_Adv_Pass_Rate', '2022-2023_Pass_Rate']

mod_df_2023['2022_2023_Adv_Pass_Rate'] = mod_df_2023['2022_2023_Adv_Pass_Rate'].str.replace('<', '-9999')
#remove??
mod_df_2023['2022_2023_Adv_Pass_Rate'] = mod_df_2023['2022_2023_Adv_Pass_Rate'].fillna(value='-9999')

mod_df_2023['2022_2023_Pass_Rate'] = mod_df_2023['2022_2023_Pass_Rate'].str.replace('<', '-9999')
#remove??
mod_df_2023['2022_2023_Pass_Rate'] = mod_df_2023['2022_2023_Pass_Rate'].fillna(value='-9999')
mod_df_2023['2022_2023_Pass_Rate'].unique()

mod_df_2023['2022_2023_Pass_Rate'] = pd.to_numeric(mod_df_2023['2022_2023_Pass_Rate'], errors='coerce').astype('int')

mod_df_2023['2022_2023_Adv_Pass_Rate'] = pd.to_numeric(mod_df_2023['2022_2023_Adv_Pass_Rate'], errors='coerce').astype('int')

In [10]:
df_2022 = pd.read_csv('https://raw.githubusercontent.com/Williamsrandrew86/SOL-data-project/main/data/school-by-test-2022.csv')

In [11]:
df_2022

,LEVEL,Div Num,Div Name,Sch Num,Sch Name,Sch Type,Low Grade,High Grade,Subject,Grade,Test,2019-2020 Pass Rate,2020-2021 Pass Rate,2021-2022 Pass Rate,2019-2020 Adv Pass Rate,2020-2021 Adv Pass Rate,2021-2022 Adv Pass Rate
0,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,English: Reading,Gr 3,English Reading,NaN,50,67,NaN,3,7
1,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,English: Reading,Gr 4,English Reading,NaN,56,67,NaN,6,11
2,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,English: Reading,Gr 5,English Reading,NaN,45,68,NaN,7,7
3,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,History and Social Sciences,Gr 5,History,NaN,<,NaN,NaN,<,NaN
4,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,History and Social Sciences,Content Specific,VA Studies,NaN,NaN,72,NaN,NaN,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26208,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Mathematics,Gr 6,Mathematics,NaN,<,NaN,NaN,<,NaN
26209,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Mathematics,Gr 7,Mathematics,NaN,<,NaN,NaN,<,NaN
26210,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Mathematics,Gr 8,Mathematics,NaN,<,NaN,NaN,<,NaN
26211,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Science,Gr 8,Science,NaN,<,NaN,NaN,<,NaN


In [12]:
mod_df_2022 = df_2022.drop(['2019-2020 Pass Rate','2020-2021 Pass Rate','2019-2020 Adv Pass Rate','2020-2021 Adv Pass Rate'], axis=1)

In [13]:
mod_df_2022.columns = mod_df_2022.columns.str.replace(' ', '_') #change the spaces with underscores
mod_df_2022.columns = mod_df_2022.columns.str.replace('-', '_') #change the hypens with underscores

In [14]:
# Specify columns to replace NaN with -9999
columns_to_replace = ['High_Grade']

# Replace NaN values with -9999 for specific columnsd
mod_df_2022[columns_to_replace] = mod_df_2022[columns_to_replace].fillna(value=-9999)

In [15]:
# Specify columns to replace NaN with -9999
columns_to_replace = ['Low_Grade']

# Replace NaN values with None for specific columns
mod_df_2022[columns_to_replace] = mod_df_2022[columns_to_replace].fillna(value='None')

In [16]:
#['2022-2023_Adv_Pass_Rate', '2022-2023_Pass_Rate']

mod_df_2022['2021_2022_Adv_Pass_Rate'] = mod_df_2022['2021_2022_Adv_Pass_Rate'].str.replace('<', '-9999')
#remove??
mod_df_2022['2021_2022_Adv_Pass_Rate'] = mod_df_2022['2021_2022_Adv_Pass_Rate'].fillna(value='-9999')

mod_df_2022['2021_2022_Pass_Rate'] = mod_df_2022['2021_2022_Pass_Rate'].str.replace('<', '-9999')
#remove??
mod_df_2022['2021_2022_Pass_Rate'] = mod_df_2022['2021_2022_Pass_Rate'].fillna(value='-9999')
mod_df_2022['2021_2022_Pass_Rate'].unique()

mod_df_2022['2021_2022_Pass_Rate'] = pd.to_numeric(mod_df_2022['2021_2022_Pass_Rate'], errors='coerce').astype('int')

mod_df_2022['2021_2022_Adv_Pass_Rate'] = pd.to_numeric(mod_df_2022['2021_2022_Adv_Pass_Rate'], errors='coerce').astype('int')

In [17]:
mod_df_2022

,LEVEL,Div_Num,Div_Name,Sch_Num,Sch_Name,Sch_Type,Low_Grade,High_Grade,Subject,Grade,Test,2021_2022_Pass_Rate,2021_2022_Adv_Pass_Rate
0,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,English: Reading,Gr 3,English Reading,67,7
1,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,English: Reading,Gr 4,English Reading,67,11
2,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,English: Reading,Gr 5,English Reading,68,7
3,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,History and Social Sciences,Gr 5,History,-9999,-9999
4,SCH,1,Accomack County,701,Accawmacke Elementary,Elem,PK,5,History and Social Sciences,Content Specific,VA Studies,72,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26208,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Mathematics,Gr 6,Mathematics,-9999,-9999
26209,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Mathematics,Gr 7,Mathematics,-9999,-9999
26210,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Mathematics,Gr 8,Mathematics,-9999,-9999
26211,SCH,98,York County,60,Yorktown Middle,Middle,06,8,Remote Science,Gr 8,Science,-9999,-9999


In [18]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/rwilliams/sol_data_project/sol_key.json"

In [19]:
client = bigquery.Client()

In [20]:
def create_or_append_table_in_bigquery(dataframe, project_id, dataset_id, table_id):
    client = bigquery.Client(project=project_id)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    try:
        table = client.get_table(table_ref)
    except:
        print(f"Table {table_id} not found in dataset {dataset_id}, creating a new one.")
        schema = [
            bigquery.SchemaField('LEVEL', 'STRING'),
            bigquery.SchemaField('Div_Num', 'INTEGER'),
            bigquery.SchemaField('Div_Name', 'STRING'),
            bigquery.SchemaField('Sch_Num', 'INTEGER'),
            bigquery.SchemaField('Sch_Name', 'STRING'),
            bigquery.SchemaField('Sch_Type', 'STRING'),
            bigquery.SchemaField('Low_Grade', 'STRING'),
            bigquery.SchemaField('High_Grade', 'INTEGER'),
            bigquery.SchemaField('Subject', 'STRING'),
            bigquery.SchemaField('Grade', 'STRING'),
            bigquery.SchemaField('Test', 'STRING'),
            bigquery.SchemaField('2022_2023_Pass_Rate', 'INTEGER'),
            bigquery.SchemaField('2022_2023_Adv_Pass_Rate', 'INTEGER')
        ]
        table = bigquery.Table(table_ref)
        table = client.create_table(table)
        print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    client.load_table_from_dataframe(dataframe, table_ref, job_config=job_config).result()

In [21]:
create_or_append_table_in_bigquery(mod_df_2023, 'valiant-surfer-411804', 'sol_data', '2023_sol_data')

Table 2023_sol_data not found in dataset sol_data, creating a new one.
Created table valiant-surfer-411804.sol_data.2023_sol_data


In [22]:
create_or_append_table_in_bigquery(mod_df_2022, 'valiant-surfer-411804', 'sol_data', '2022_sol_data')

Table 2022_sol_data not found in dataset sol_data, creating a new one.
Created table valiant-surfer-411804.sol_data.2022_sol_data
